In [123]:
import jax
import jax.numpy as jnp
import jax.random as random
from jax.numpy.fft import fft, ifft, fftn, ifftn
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
@jax.jit
def circ_mult(w,x):
    return jnp.real(fft(ifft(x.T)*fft(w)).T)

circ_vmap = jax.vmap(lambda col, ind: jnp.roll(col, ind), in_axes=(None,0), out_axes=1)

@jax.jit
def reg_mult(w,x):
    C = circ_vmap(w.reshape(1,-1), jnp.arange(D_Z))[0]
    return C @ x


In [103]:
C[:,0]

Array([ 1.62264216e+00, -8.18689287e-01, -2.54764885e-01, -1.55617642e+00,
        1.04647315e+00,  7.28739873e-02, -6.66688561e-01,  1.12391198e+00,
       -1.72950840e+00, -1.64727375e-01, -2.74157941e-01,  1.13316286e+00,
        1.25017092e-01, -2.86463916e-01, -1.19328606e+00,  1.91607726e+00,
       -4.74351019e-01, -1.56128228e+00, -1.11661828e+00,  1.96414077e+00,
       -8.04406777e-03,  1.01498105e-01, -1.82441995e-01,  6.77530110e-01,
        4.72201049e-01,  8.55871812e-02, -1.60682452e+00,  1.41883627e-01,
        4.40597296e-01,  5.82074165e-01,  7.95560360e-01,  5.53778887e-01,
        1.00075707e-01,  5.47479868e-01, -9.17235196e-01,  5.82158938e-03,
       -3.28320116e-01,  4.49487656e-01,  4.10299897e-02, -1.34911144e+00,
       -4.65559602e-01, -6.59652352e-01, -1.50986416e-02,  2.33468676e+00,
       -2.53834738e-03,  4.37291175e-01, -2.56920576e-01, -6.96839750e-01,
       -8.16014290e-01, -4.18101907e-01, -1.50813270e+00,  9.66534257e-01,
       -1.11704206e+00,  

In [104]:
w

Array([ 1.62264216e+00,  2.02526474e+00, -4.33594435e-01, -7.86173493e-02,
        1.76090896e-01, -9.72089231e-01, -4.95298743e-01,  4.94378597e-01,
        6.64349318e-01, -9.50163484e-01,  2.17953038e+00, -1.95515060e+00,
        3.58570725e-01,  1.57795131e-01,  1.27708471e+00,  1.51046479e+00,
        9.70655978e-01,  5.99608064e-01,  2.47007050e-02, -1.91647720e+00,
       -1.85934913e+00,  1.72814405e+00,  4.71903495e-02,  8.14127982e-01,
        1.31327674e-01,  2.82847047e-01,  1.24359429e+00,  6.90280080e-01,
       -8.00737441e-01, -7.40989983e-01, -1.53882873e+00,  3.02691847e-01,
       -2.07160451e-02,  1.13287210e-01, -2.20654696e-01,  7.05225617e-02,
        8.53295803e-01, -8.21773827e-01, -1.46142114e-02, -1.50462165e-01,
       -9.00135219e-01, -7.59072721e-01,  3.33095133e-01,  8.09249043e-01,
        4.26925533e-02, -5.77671230e-01, -4.14398938e-01, -1.94125330e+00,
        1.31611836e+00,  7.54272819e-01,  1.61709309e-01, -3.48330699e-02,
       -1.33064091e+00,  

In [95]:
D_Z = 500
N = 1
w_warm = jax.random.normal(random.PRNGKey(0), (D_Z,))
x_warm = jax.random.normal(random.PRNGKey(1), (N,D_Z))
w = jax.random.normal(random.PRNGKey(0), (D_Z,))
x = jax.random.normal(random.PRNGKey(1), (N,D_Z))
C_warm = circ_vmap(w_warm.reshape(1,-1), jnp.arange(D_Z))[0]
C_warm @ x_warm.T
C_warm @ x_warm.T
C_warm @ x_warm.T
time_start = time.time()
C = circ_vmap(w.reshape(1,-1), jnp.arange(D_Z))[0]
jnp.matrix_transpose(C @ x.T) # to get x @ C'
time_end = time.time()
print(f"Time: {time_end - time_start:.4f}")
# Z = (N, D_Z)
# W = (D_Z, D_Z)
# WZ = (Z^T @ W^T)^T


Time: 0.0051


In [94]:
jnp.real(circ_mult(w_warm, x_warm.T))
jnp.real(circ_mult(w_warm, x_warm.T))
jnp.real(circ[0]ime.time()
print(f"Time: {time_end - time_start:.4f}")

Time: 0.0002


In [72]:
circ_grad = jax.jacobian(circ_mult)
circ_grad(w_warm, x_warm.T)
circ_grad(w_warm, x_warm.T)
circ_grad(w_warm, x_warm.T)
time_start = time.time()
res = circ_grad(w, x.T)
time_end = time.time()
print(f"Time: {time_end - time_start:.4f}. Res shape: {res.shape}")

Time: 0.0071. Res shape: (500, 1, 500)


In [70]:
mult_grad = jax.jacobian(reg_mult)
mult_grad(w_warm, x_warm.T)
mult_grad(w_warm, x_warm.T)
mult_grad(w_warm, x_warm.T)
time_start = time.time()
res = mult_grad(w, x.T)
time_end = time.time()
print(f"Time: {time_end - time_start:.4f}. Res shape: {res.shape}")

Time: 0.5629. Res shape: (500, 1, 500)


In [4]:
reg_mult(w, x.T)

Array([[ 4.59632969e+00],
       [-1.41770697e+01],
       [-1.21102448e+01],
       [ 1.31331644e+01],
       [-3.50147400e+01],
       [-8.32195663e+00],
       [ 4.55397301e+01],
       [-2.67613316e+01],
       [ 1.38328876e+01],
       [-4.97577286e+01],
       [ 1.30531139e+01],
       [ 1.91941700e+01],
       [-4.72279320e+01],
       [ 6.16238642e+00],
       [ 1.93773136e+01],
       [-9.37050247e+00],
       [-2.81911125e+01],
       [ 8.06140900e+00],
       [ 8.72234631e+00],
       [-1.39772310e+01],
       [ 2.52091484e+01],
       [-3.29163589e+01],
       [ 5.11376572e+00],
       [ 4.49327707e+00],
       [-2.97822895e+01],
       [ 1.39635248e+01],
       [ 2.20934982e+01],
       [ 1.06957781e+00],
       [ 8.10378361e+00],
       [-3.91817398e+01],
       [-9.13887215e+00],
       [-4.08212051e+01],
       [ 1.02809870e+00],
       [ 2.31028538e+01],
       [ 1.64405537e+01],
       [ 1.43364449e+01],
       [-6.41580534e+00],
       [-1.76827488e+01],
       [ 5.0

In [5]:
circ_mult(w, x.T)

Array([[ 4.59633446e+00],
       [-1.41770744e+01],
       [-1.21102438e+01],
       [ 1.31331625e+01],
       [-3.50147285e+01],
       [-8.32195663e+00],
       [ 4.55397453e+01],
       [-2.67613525e+01],
       [ 1.38328876e+01],
       [-4.97577477e+01],
       [ 1.30531197e+01],
       [ 1.91941643e+01],
       [-4.72279396e+01],
       [ 6.16239262e+00],
       [ 1.93773117e+01],
       [-9.37049675e+00],
       [-2.81911316e+01],
       [ 8.06141663e+00],
       [ 8.72235203e+00],
       [-1.39772453e+01],
       [ 2.52091389e+01],
       [-3.29163513e+01],
       [ 5.11376476e+00],
       [ 4.49327087e+00],
       [-2.97822952e+01],
       [ 1.39635162e+01],
       [ 2.20935059e+01],
       [ 1.06958675e+00],
       [ 8.10378075e+00],
       [-3.91817398e+01],
       [-9.13887024e+00],
       [-4.08212051e+01],
       [ 1.02809334e+00],
       [ 2.31028652e+01],
       [ 1.64405594e+01],
       [ 1.43364525e+01],
       [-6.41579485e+00],
       [-1.76827469e+01],
       [ 5.0

In [32]:
C.shape

(100, 100)

In [35]:
ifft(fft(w) * fft(x.T))[0,0]

Array(-0.2505963+2.384186e-09j, dtype=complex64)

# Computation of $Cx$ using DFT
Cheng et al claims that
$$Cx=\mathcal{F}^{-1}(\mathcal{F}(r) \circ \mathcal{F}(x))$$
However, this does not work (see below)
I have found a solution that does work though (based on https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf)
$$Cx=\mathcal{F}(\mathcal{F}(r) \circ \mathcal{F}^{-1}(x))$$

In [121]:
R = jnp.array([[1,3,2],[2,1,3],[3,2,1]])
r = jnp.array([1,2,3])
x = jnp.array([[4,5,6], [7,8,9], [10,11,12]]).T
print(R)
print(r)
print(x)

[[1 3 2]
 [2 1 3]
 [3 2 1]]
[1 2 3]
[[ 4  7 10]
 [ 5  8 11]
 [ 6  9 12]]


In [161]:
print("Reference", (R @ x))
print("Cheng et al.", jnp.real(ifftn(fft(r) * fftn(x, axes=(0,)).T, axes=(1,)).T))
print("Reference, transposed", (R.T @ x))
print("My fix", jnp.real(fft(fft(r) * ifft(x))))

Reference [[31 49 67]
 [31 49 67]
 [28 46 64]]
Cheng et al. [[31. 49. 67.]
 [31. 49. 67.]
 [28. 46. 64.]]
Reference, transposed [[32 50 68]
 [29 47 65]
 [29 47 65]]
My fix [[48. 39. 39.]
 [54. 45. 45.]
 [60. 51. 51.]]


In [114]:
jnp.real(fft(fft(w) * ifft(x[0]))[:3])

ValueError: axis -1 is out of bounds for array of dimension 0

In [192]:
def fft_mult(r, x):
    return jnp.real(ifft(fft(r) * fft(x)))
fft_jac = jax.jacobian(lambda r, x: fft_mult(r, x))

In [188]:
fft_grad(r.astype(jnp.float32),x.astype(jnp.float32)[:,0])

Array([4., 6., 5.], dtype=float32)

In [189]:
jax.grad(lambda x: jnp.real(ifft(x)[0]))(fft(r) * fft(x[0]))[0] @ ifft(x[0])[0]

ValueError: matmul input operand 0 must have ndim at least 1, but it has ndim 0

In [193]:
jax.make_jaxpr(fft_jac)(r.astype(jnp.float32),x.astype(jnp.float32)[:,0])

{ lambda ; a:f32[3] b:f32[3]. let
    c:c64[3] = pjit[
      name=fft
      jaxpr={ lambda ; d:f32[3]. let
          e:c64[3] = convert_element_type[new_dtype=complex64 weak_type=False] d
          f:c64[3] = fft[fft_lengths=(3,) fft_type=0] e
        in (f,) }
    ] a
    g:c64[3] = pjit[
      name=fft
      jaxpr={ lambda ; h:f32[3]. let
          i:c64[3] = convert_element_type[new_dtype=complex64 weak_type=False] h
          j:c64[3] = fft[fft_lengths=(3,) fft_type=0] i
        in (j,) }
    ] b
    k:c64[3] = mul c g
    l:c64[3] = pjit[
      name=fft
      jaxpr={ lambda ; m:c64[3]. let
          n:c64[3] = fft[fft_lengths=(3,) fft_type=1] m
        in (n,) }
    ] k
    _:f32[3] = real l
    o:i32[3,3] = iota[dimension=0 dtype=int32 shape=(3, 3) sharding=None] 
    p:i32[3,3] = iota[dimension=1 dtype=int32 shape=(3, 3) sharding=None] 
    q:i32[3,3] = add o 0
    r:bool[3,3] = eq q p
    s:f32[3,3] = convert_element_type[new_dtype=float32 weak_type=False] r
    t:f32[3,3] = sp